#  <span style="color:orange">Topic Modeling con NLP</span>

Fuente: https://pycaret.org

# 2.0 ¿Qué es el procesamiento del lenguaje natural?

El procesamiento del lenguaje natural (PNL) es una rama de la inteligencia artificial que se ocupa de analizar, comprender y generar los lenguajes que los humanos usan de forma natural para interactuar con las computadoras en contextos tanto escritos como hablados utilizando lenguajes humanos naturales en lugar de lenguajes informáticos. Algunos de los casos de uso común de la PNL en el aprendizaje automático son:

- **Descubrimiento y modelado de temas:** Capture el significado y los temas en colecciones de texto y aplique técnicas avanzadas de modelado como Modelado de temas para agrupar documentos similares.
- **Análisis de sentimientos:** Identificar el estado de ánimo o las opiniones subjetivas dentro de grandes cantidades de texto, incluido el sentimiento promedio y la minería de opiniones.
- **Resumen de documentos:** Generación automática de sinopsis de grandes cuerpos de texto.
- **Conversión de voz a texto y de texto a voz:** Transformación de comandos de voz en texto escrito y viceversa.
- **Traducción automática:** Traducción automática de texto o voz de un idioma a otro.


# 3.0 Descripción general del módulo de procesamiento de lenguaje natural en PyCaret
El módulo NLP de PyCaret (`pycaret.nlp`) es un módulo de aprendizaje automático no supervisado que se puede utilizar para analizar los datos de texto creando un modelo de tema para encontrar la estructura semántica oculta en los documentos. El módulo NLP de PyCaret viene integrado con una amplia gama de técnicas de preprocesamiento de texto, que es el paso fundamental en cualquier problema de NLP. Transforma el texto sin procesar en un formato del que pueden aprender los algoritmos de aprendizaje automático.

A partir de la primera versión, el módulo NLP de PyCaret solo admite el idioma inglés y proporciona varias implementaciones populares de modelos de temas desde la asignación de Dirichlet latente hasta la factorización de matrices no negativas. Tiene más de 5 algoritmos listos para usar y más de 10 gráficos para analizar el texto. El módulo NLP de PyCaret también implementa una función única `tune_model ()` que le permite ajustar los hiperparámetros de un modelo de tema para optimizar el objetivo de aprendizaje supervisado, como ʻAUC` para clasificación o `R2` para regresión.

# 4.0 Dataset

Para este tutorial usaremos datos de **Kiva Microfunds** https://www.kiva.org/. Kiva Microfunds es una organización sin fines de lucro que permite a las personas prestar dinero a empresarios y estudiantes de bajos ingresos de todo el mundo. Desde su inicio en 2005, Kiva ha financiado colectivamente millones de préstamos con una tasa de reembolso de alrededor del 98%. En Kiva, cada solicitud de préstamo incluye información demográfica tradicional sobre el prestatario, como el sexo y la ubicación, así como una historia personal. En este tutorial usaremos el texto proporcionado en la historia personal para obtener información sobre el conjunto de datos y comprender la estructura semántica oculta en el texto. El conjunto de datos contiene 6818 muestras. A continuación se muestra una breve descripción de las características:

- **país:** país del prestatario
- **en:** Historia personal del prestatario cuando solicitó el préstamo
- **género:** género (M = masculino, F = femenino)
- **cantidad_préstamo:** Cantidad de préstamo aprobado y desembolsado
- **impago:** Tipo de prestamista (Prestamista = usuario personal registrado en el sitio web de Kiva, Socio = institución de microfinanzas que trabaja con Kiva para encontrar y financiar préstamos)
- **sector:** sector del prestatario
- **estado:** estado del préstamo (1-incumplimiento, 0-reembolsado)



#### Reconocimiento del conjunto de datos:
Kiva Microfunds https://www.kiva.org/

In [ ]:
!pip install pycaret

# 5.0 Obtención de la información

In [8]:
from pycaret.datasets import get_data
data = get_data('kiva')

,country,en,gender,loan_amount,nonpayment,sector,status
0,Dominican Republic,"""Banco Esperanza"" is a group of 10 women looki...",F,1225,partner,Retail,0
1,Dominican Republic,"""Caminemos Hacia Adelante"" or ""Walking Forward...",F,1975,lender,Clothing,0
2,Dominican Republic,"""Creciendo Por La Union"" is a group of 10 peop...",F,2175,partner,Clothing,0
3,Dominican Republic,"""Cristo Vive"" (""Christ lives"" is a group of 10...",F,1425,partner,Clothing,0
4,Dominican Republic,"""Cristo Vive"" is a large group of 35 people, 2...",F,4025,partner,Food,0


In [9]:
#check the shape of data
data.shape

(6818, 7)

In [10]:
# sampling the data to select only 1000 documents
data = data.sample(1000, random_state=786).reset_index(drop=True)
data.shape

(1000, 7)

# 6.0 Configuración del entorno en PyCaret

La función `setup()` inicializa el entorno en pycaret y realiza varios pasos de preprocesamiento de texto que son imperativos para trabajar con problemas de PNL. setup debe ser llamado antes de ejecutar cualquier otra función en pycaret. Se necesitan dos parámetros: el marco de datos de pandas y el nombre de la columna de texto pasada como parámetro `target`. También puede pasar una `lista` que contenga texto, en cuyo caso no es necesario pasar el parámetro `objetivo`. Cuando se ejecuta la configuración, los siguientes pasos de preprocesamiento se aplican automáticamente:

- **Eliminación de caracteres numéricos:** Todos los caracteres numéricos se eliminan del texto. Se reemplazan con espacios en blanco. <br/>
<br/>
- **Eliminación de caracteres especiales:** Todos los caracteres especiales no alfanuméricos se eliminan del texto. También se reemplazan con espacios en blanco. <br/>
<br/>
- **Tokenización de palabras:** La tokenización de palabras es el proceso de dividir una gran muestra de texto en palabras. Este es el requisito fundamental en las tareas de procesamiento del lenguaje natural, donde cada palabra debe capturarse por separado para su posterior análisis.  <br/>
<br/>
- **Eliminación de palabras vacías:** Una palabra vacía (o palabra inexistente) es una palabra que a menudo se elimina del texto porque es común y proporciona poco valor para la recuperación de información, aunque puede ser lingüísticamente significativa. Ejemplos de tales palabras en inglés son: "the", "a", "an", "in", etc.<br / >
<br/>
- **Extracción de bigrama:** Un bigrama es una secuencia de dos elementos adyacentes de una cadena de tokens, que suelen ser letras, sílabas o palabras. Por ejemplo: la palabra Nueva York se captura como dos palabras diferentes "Nueva" y "York" cuando se realiza la tokenización, pero si se repite las veces suficientes, Bigram Extraction representará la palabra como una, es decir, "New_York"  <br/>
<br/>

- **Extracción de trigram:** Similar a la extracción de bigrama, el trigram es una secuencia de tres elementos adyacentes de una cadena de tokens. <br/>
<br/>
- **Lematización: **La lematización es el proceso de agrupar las formas flexionadas de una palabra para que puedan analizarse como una sola palabra, identificada por el lema de la palabra o la forma del diccionario. En el idioma inglés, la palabra aparece en varias formas flexionadas. Por ejemplo, el verbo "caminar" puede aparecer como "caminar", "caminar", "caminar", "caminar". La forma básica, 'caminar', que uno podría buscar en un diccionario, se llama lema de la palabra<br/>
<br/>
- **Palabras irrelevantes personalizadas:** Muchas veces el texto contiene palabras que no son palabras irrelevantes según la regla del idioma, pero que añaden poca o ninguna información. Por ejemplo, en este tutorial estamos usando el conjunto de datos de préstamos. Como tal, palabras como "préstamo", "banco", "dinero", "negocio" son demasiado obvias y no añaden valor. La mayoría de las veces, también agregan mucho ruido en el modelo de tema. Puede eliminar esas palabras del corpus utilizando el parámetro `custom_stopwords`.

**Nota:** Algunas funcionalidades en `pycaret.nlp` requieren un modelo de idioma inglés. El modelo de idioma no se descarga automáticamente cuando instala pycaret. Tendrá que descargar esta interfaz de línea de comandos de Python, como Anaconda Prompt. Para descargar el modelo, escriba lo siguiente en su línea de comando:

`python -m spacy descargar en_core_web_sm` <br/>
`python -m textblob.download_corpora` <br/>

In [11]:
from pycaret.nlp import *
exp_nlp101 = setup(data = data, target = 'en', session_id = 123)

INFO:logs:PyCaret NLP Module
INFO:logs:version 2.1.2
INFO:logs:Initializing setup()
INFO:logs:USI: abfd
INFO:logs:setup(data=(1000, 7), target=en, custom_stopwords=None, html=True, session_id=123, log_experiment=False,
                    experiment_name=None, log_plots=False, log_data=False, verbose=True)
INFO:logs:Checking environment
INFO:logs:python_version: 3.7.6
INFO:logs:python_build: ('default', 'Jan  8 2020 20:23:39')
INFO:logs:machine: AMD64
INFO:logs:platform: Windows-10-10.0.18362-SP0
INFO:logs:Memory: svmem(total=17059516416, available=8020533248, percent=53.0, used=9038983168, free=8020533248)
INFO:logs:Physical Core: 4
INFO:logs:Logical Core: 8
INFO:logs:Checking libraries
INFO:logs:pd==1.1.1
INFO:logs:numpy==1.18.1
INFO:logs:gensim==3.8.3
INFO:logs:nltk==3.5
INFO:logs:textblob==0.15.3
INFO:logs:pyLDAvis==2.1.2
INFO:logs:wordcloud==1.7.0
INFO:logs:mlflow==1.11.0
INFO:logs:Checking Exceptions
ERROR:root:Internal Python error in the inspect module.
Below is the traceback f

Traceback (most recent call last):
  File "C:\Users\Leyre\anaconda3\lib\site-packages\pycaret\nlp.py", line 283, in setup
    import spacy
  File "C:\Users\Leyre\anaconda3\lib\site-packages\spacy\__init__.py", line 12, in <module>
    from . import pipeline
  File "C:\Users\Leyre\anaconda3\lib\site-packages\spacy\pipeline\__init__.py", line 4, in <module>
    from .pipes import Tagger, DependencyParser, EntityRecognizer, EntityLinker
  File "pipes.pyx", line 1, in init spacy.pipeline.pipes
  File "stringsource", line 105, in init spacy.syntax.nn_parser
AttributeError: type object 'spacy.syntax.nn_parser.array' has no attribute '__reduce_cython__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Leyre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-3d7c5fbfb5c1>", line 2, in <module>
    exp_nlp101 

TypeError: object of type 'NoneType' has no len()

Una vez que la instalación se ejecuta con éxito, imprime la cuadrícula de información con la siguiente información:

- **session_id:** Un número pseduo-aleatorio distribuido como semilla en todas las funciones para su posterior reproducibilidad. Si no se pasa `session_id`, se genera automáticamente un número aleatorio que se distribuye a todas las funciones. En este experimento, session_id se establece como `123` para su posterior reproducibilidad. <br/>
<br/>
- **Documentos:** Número de documentos (o muestras en el conjunto de datos si se pasa el marco de datos). <br/>
<br/>
- **Tamaño de vocabulario:** Tamaño del vocabulario en el corpus después de aplicar todo el preprocesamiento del texto, como la eliminación de palabras vacías, extracción de bigramas / trigramas, lematización, etc. <br/>

Observe que todos los pasos de preprocesamiento de texto se realizan automáticamente cuando ejecuta `setup ()`. Estos pasos son imprescindibles para realizar cualquier experimento de PNL. La función `setup ()` prepara el corpus y el diccionario que está listo para usar para los modelos de tema que puede crear usando la función `create_model ()`. Otra forma de pasar el texto es en forma de lista, en cuyo caso no se necesita el parámetro `target`.

# 7.0 Crear un modelo de tema

**¿Qué es el modelo de tema?** En el aprendizaje automático y el procesamiento del lenguaje natural, un modelo de tema es un tipo de modelo estadístico para descubrir los "temas" abstractos que ocurren en una colección de documentos. El modelado de temas es una herramienta de extracción de texto de uso frecuente para el descubrimiento de estructuras semánticas ocultas en un cuerpo de texto. Intuitivamente, dado que un documento trata sobre un tema en particular, uno esperaría que aparecieran palabras específicas en el documento con más o menos frecuencia: "perro" y "hueso" aparecerán con más frecuencia en documentos sobre perros, "gato" y "miau". aparecerá en documentos sobre gatos, y "el" y "es" aparecerán igualmente en ambos. Por lo general, un documento se refiere a múltiples temas en diferentes proporciones; por lo tanto, en un documento que trata sobre gatos en un 10% y sobre perros en un 90%, probablemente habría alrededor de 9 veces más palabras de perros que de gatos. Los "temas" producidos por las técnicas de modelado de temas son grupos de palabras similares. Un modelo de tema captura esta intuición en un marco matemático, que permite examinar un conjunto de documentos y descubrir, con base en las estadísticas de las palabras en cada uno, cuáles podrían ser los temas y cuál es el balance de temas de cada documento. 

Crear un modelo de tema en PyCaret es simple y similar a cómo habría creado un modelo en módulos supervisados ​​de pycaret. Se crea un modelo de tema usando la función `create_model ()` que toma un parámetro obligatorio, es decir, el nombre del modelo como una cadena. Esta función devuelve un objeto de modelo entrenado. Hay 5 modelos de temas disponibles en PyCaret. vea la cadena de documentos de `create_model ()` para una lista completa de modelos. Vea un ejemplo a continuación donde creamos el modelo de asignación de Dirichlet latente (LDA):

In [ ]:
lda = create_model('lda')

In [ ]:
print(lda)

Hemos creado el modelo de asignación de Dirichlet latente (LDA) con una sola palabra, es decir, `create_model ()`. Observe que el parámetro `num_topics` está establecido en` 4`, que es un valor predeterminado que se toma cuando no pasa el parámetro `num_topics` en` create_model () `. En el siguiente ejemplo, crearemos un modelo LDA con 6 temas y también estableceremos el parámetro `multi_core` en` True`. Cuando `multi_core` se establece en` True`, la asignación de Dirichlet latente (LDA) utiliza todos los núcleos de CPU para paralelizar y acelerar el entrenamiento del modelo.

In [ ]:
lda2 = create_model('lda', num_topics = 6, multi_core = True)

In [ ]:
print(lda2)

# 8.0 Asignar un Model

Ahora que hemos creado un modelo de tema, nos gustaría asignar las proporciones de tema a nuestro conjunto de datos (6818 documentos / muestras) para analizar los resultados. Lo lograremos usando la función ʻassign_model ()`. Vea un ejemplo a continuación:

In [ ]:
lda_results = assign_model(lda)
lda_results.head()

Observe cómo ahora se agregan 6 columnas adicionales al marco de datos. ʻEn` es el texto después de todo el preprocesamiento. `Topic_0 ... Topic_3` son las proporciones de los temas y representan la distribución de temas para cada documento. `Dominant_Topic` es el número de tema con mayor proporción y` Perc_Dominant_Topic` es el porcentaje de tema dominante sobre 1 (solo se muestra cuando los modelos son estocásticos, es decir, la suma de todas las proporciones es igual a 1).

# 9.0 Visualizar el modelo

La función `plot_model ()` se puede usar para analizar el corpus general o solo temas específicos extraídos a través del modelo de temas. Por lo tanto, la función `plot_model ()` también puede funcionar sin pasar ningún objeto de modelo entrenado. Vea ejemplos a continuación:

### 9.1 Frequency Distribution of Entire Corpus

In [ ]:
plot_model()

### 9.2 Top 100 Bigrams on Entire Corpus

In [ ]:
plot_model(plot = 'bigram')

### 9.3 Frequency Distribution of Topic 1

`plot_model ()` también se puede usar para analizar los mismos gráficos para temas específicos. Para generar gráficos a nivel de tema, la función requiere que el objeto de modelo entrenado se pase dentro de `plot_model ()`. En el siguiente ejemplo, generaremos una distribución de frecuencia en el `Tema 1` solo como se define en el parámetro` topic_num`.

In [ ]:
plot_model(lda, plot = 'frequency', topic_num = 'Topic 1')

### 9.4 Topic Distribution

In [ ]:
plot_model(lda, plot = 'topic_distribution')

Cada documento es una distribución de temas y no un solo tema. Aunque, si la tarea es categorizar el documento en temas específicos, no sería incorrecto usar la proporción de temas con el valor más alto para categorizar el documento en **un tema**. En el gráfico anterior, cada documento se clasifica en un tema utilizando la mayor proporción de ponderaciones de temas. Podemos ver que la mayoría de los documentos están en el "Tema 3" y sólo unos pocos en el "Tema 1". Si pasa el mouse sobre estas barras, obtendrá una idea básica de los temas de este tema al mirar las palabras clave.

Por ejemplo, si evalúa el "Tema 2", verá palabras clave como "agricultor", "arroz", "tierra", lo que probablemente significa que los solicitantes de préstamos en esta categoría pertenecen a préstamos agrícolas. Sin embargo, si coloca el cursor sobre el `Tema 0` y el` Tema 3`, observará que muchas repeticiones y palabras clave se superponen en todos los temas, como la palabra "préstamo" y "negocio" que aparecen tanto en el `Tema 0` como en el` Tema 3`

### 9.5 T-distributed Stochastic Neighbor Embedding (t-SNE)

In [ ]:
plot_model(lda, plot = 'tsne')

La incrustación de vecinos estocásticos distribuidos en T (t-SNE) es una técnica de reducción de dimensionalidad no lineal muy adecuada para incrustar datos de alta dimensión para visualización en un espacio de baja dimensión de dos o tres dimensiones.

### 9.6 Uniform Manifold Approximation and Projection Plot

In [ ]:
plot_model(lda, plot = 'umap')

UMAP (Uniform Manifold Approximation and Projection) es una novedosa técnica de aprendizaje múltiple para la reducción de dimensionalidad. Es similar a tSNE y PCA en su propósito, ya que todas son técnicas para reducir la dimensionalidad para proyecciones 2d / 3d. UMAP se construye a partir de un marco teórico basado en la geometría riemanniana y la topología algebraica.

# 10.0 Evaluar el Modelo

Otra forma de analizar el rendimiento de los modelos es usar la función ʻevaluate_model () `que muestra una interfaz de usuario para todos los gráficos disponibles para un modelo dado. Utiliza internamente la función `plot_model ()`. Vea el ejemplo a continuación en el que hemos generado un gráfico de polaridad de sentimiento para el `Tema 3` utilizando el modelo LDA almacenado en la variable` lda`.

In [ ]:
evaluate_model(lda)

# 11.0 Guardar el modelo

A medida que profundice en el procesamiento del lenguaje natural, aprenderá que el tiempo de entrenamiento de los modelos de temas aumenta exponencialmente a medida que aumenta el tamaño del corpus. Como tal, si desea continuar con su experimento o análisis en un momento posterior, no necesita repetir todo el experimento y volver a entrenar su modelo. La función incorporada de PyCaret `save_model ()` le permite guardar el modelo para su uso posterior.

In [ ]:
save_model(lda,'Final LDA Model 08Feb2020')

# 12.0 Cargar el modelo

Para cargar un modelo guardado en una fecha futura en el mismo entorno o en otro diferente, usaríamos la función `load_model ()` de PyCaret.

In [ ]:
saved_lda = load_model('Final LDA Model 08Feb2020')

In [ ]:
print(saved_lda)